In [1]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
from datetime import date
import pandas as pd

#Define global variables related to Sentinel-1
sentinel2Info = {
  "collection": "COPERNICUS/S2_SR_HARMONIZED",
  "startYear" : "2014-04-01",
  "endDate"   : None
}



## method that returns the name of the collection used in the Sentinel1 class
def getSentinel2Col():
    return sentinel2Info["collection"]

## method that sets the Sentinel1 collection 
#  this is useful in case an updated class is released in GEE or for consistency an 
#  older version of Sentinel1 col needs to be used
# @param[in] newcol: the name of the collection that will replace the collection 
# used in this class
# @note: Please use with caution as a wrong collection will crash the entire class
def setSentinel2Col(newCol):
    global sentinel2Col
    sentinel2Info["collection"] = newCol


## method that returns the the start end end date of the collection
#  @param[in] type 'whole' or 'year' get the entire date or just the year
def getStartEndDateS2(type):
  startDate = sentinel2Info["startYear"]
  endDate   = sentinel2Info["endDate"  ]
  if(endDate == None):
    today = date.today()
    endDate = today.strftime("%Y-%m-%d")
  if type == 'year' :
     return [getYearMonthOrDay(startDate,'year'),getYearMonthOrDay(endDate,'year')]
  elif type == 'whole':
     return [startDate,endDate]
  else: # type not defined
     raise Exception("ERROR: type for getting start end end date/year of Sentinel-1 not defined\n")
     

In [3]:
import sys
import indices
from indices import INDEX_NAMES



# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules: 
   # import GEE and Authenticate, token or log in will be asked from web browser
   import ee
   ee.Authenticate()
   ee.Initialize()
#else: 
   # google earth engine already imported and authenticated
  

modulename = 'ipynb_masks'
if modulename not in sys.modules:
    %run Masks.ipynb
    sys.modules['ipynb_masks'] = None
#else
    # module already loaded

modulename = 'ipynb_Utils'
if modulename not in sys.modules:
    %run Utils.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Utils'] = None 
#else
   # Utils modules has already been loaded somewhere else

modulename = 'ipynb_masks'
if modulename not in sys.modules:
    %run Masks.ipynb 
    sys.modules['ipynb_masks'] = None
#else
    # module already loaded

Utils imported
Masks class imported


In [4]:
# Public info for printing the available Indices of Sentinel-2 and the relevant information

indiceKeys = INDEX_NAMES
"""
IndexName  = ['Normalised Difference Vegetation Index']
formula    = ['(B8 - B4) / (B8 + B4)']

availableIndices = pd.DataFrame(data={
        "Key": indiceKeys, 
        "Index": IndexName, 
        "Formula": formula}
        ) 


        
def printAvailableIndicesS2():
    print(availableIndices)
"""

'\nIndexName  = [\'Normalised Difference Vegetation Index\']\nformula    = [\'(B8 - B4) / (B8 + B4)\']\n\navailableIndices = pd.DataFrame(data={\n        "Key": indiceKeys, \n        "Index": IndexName, \n        "Formula": formula}\n        ) \n\n\n        \ndef printAvailableIndicesS2():\n    print(availableIndices)\n'

In [ ]:
class Sentinel2:
   
    def add_cloud_bands(self,img):
        # Get s2cloudless image, subset the probability band.
        cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

        # Condition s2cloudless by the probability threshold value.
        is_cloud = cld_prb.gt(self.CLD_PRB_THRESH).rename('clouds')

        # Add the cloud probability layer and cloud mask as image bands.
        return img.addBands(ee.Image([cld_prb, is_cloud]))

    def add_shadow_bands(self,img):
        # Identify water pixels from the SCL band.
        not_water = img.select('SCL').neq(6)

        # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
        SR_BAND_SCALE = 1e4
        dark_pixels = img.select('B8').lt(self.NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

        # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
        shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

        # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
        cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, self.CLD_PRJ_DIST*10)
            .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
            .select('distance')
            .mask()
            .rename('cloud_transform'))

        # Identify the intersection of dark pixels with cloud shadow projection.
        shadows = cld_proj.multiply(dark_pixels).rename('shadows')

        # Add dark pixels, cloud projection, and identified shadows as image bands.
        return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

    def add_cld_shdw_mask(self,img):
        # Add cloud component bands.
        img_cloud = self.add_cloud_bands(img)

        # Add cloud shadow component bands.
        img_cloud_shadow = self.add_shadow_bands(img_cloud)

        # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
        is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
        # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
        is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(self.BUFFER*2/20)
            .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
            .rename('cloudmask'))

        img_cloud_shadow = img_cloud_shadow.addBands(is_cld_shdw) 
        # Add the final cloud-shadow mask to the image.
        return img_cloud_shadow

    def applyCloudMask(self,img):
        #result = img.where(img.select('cloudmask').gt(0),1)
        #resultUnmasked = result.unmask(0)
        #mask = resultUnmasked.unmask(-999).eq(-999)
        cloudmask =  img.select('cloudmask').selfMask()
        tmp1 = cloudmask.where(cloudmask.gt(0),0)
        tmp1 = tmp1.unmask(1)
        return img.updateMask(tmp1)
    
 
    def addNDVIimg(self, image):
        # NDVI = (NIR - Red) / (NIR + Red)
        ndvi = image.expression(
            '(nir - red) / (nir + red)', {
                'nir': image.select('B8'),
                'red': image.select('B4')
            } 
        )
        return image.addBands(ndvi.rename('ndvi'))
    
    # @brief initialisation of Sentinel2 class
    # @param[in] geometry: area of interest
    # @param[in] startDate: start date of the collection to be retrived
    # @param[in] endDate: end date of the collection to be retrived
    # @param[in] cloudfilter: percentage of non acceptable clouds for removing images
    # @param[in] masks: a dictonary containing the masks to be applied 
    # Format of masks should be the following, but it should include only the masks 
    # that we want to be applied
    # masks = {'gsw':buffer, 'lmask': buffer, 'forestMask': {buffer,year}}
    def __init__(self,geometry, sentinel2Info, masks):
        self.allbands       = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE', 'probability', 'clouds', 'dark_pixels', 'cloud_transform', 'shadows', 'cloudmask']
        self.selectedBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
        self.selectedIndices = []
        self.startDate      = '2019-01-01'
        self.endDate        = '2019-12-31'
        self.CLOUD_FILTER   = 50
        self.CLD_PRB_THRESH = 50
        self.NIR_DRK_THRESH = 0.15
        self.CLD_PRJ_DIST = 1
        self.BUFFER = 50
        
        if("startDate" in sentinel2Info and sentinel2Info["startDate"]!=""):
            self.startDate = str(sentinel2Info["startDate"])
            if("endDate" in sentinel2Info and sentinel2Info["endDate"]!=""):
                self.endDate = str(sentinel2Info["endDate"])
            else:
                print ("WARNING: Either startDate or endDate were not defined, default values used")
                print ("startDate: ", self.startDate, " & endDate: ", self.endDate)
        else:
            print ("WARNING: Either startDate or endDate were not defined, default values used")
            print ("startDate: ", self.startDate, " & endDate: ", self.endDate)
        
        if("selectedBand" in sentinel2Info and sentinel2Info["clouds"]!=None):
            self.CLOUD_FILTER = sentinel2Info["clouds"]
        else : 
            print("WARNING: Cloud percentage threshold not define. Default value is ", self.CLOUD_FILTER)
        
        
        if("selectedBand" in sentinel2Info and sentinel2Info["selectedBand"]!=None):
            self.selectedBands = sentinel2Info["selectedBand"]
        else : 
            print ("WARNING: Default bands are used: ", self.selectedBands)
        if("selectedIndices" in sentinel2Info and sentinel2Info["selectedIndices"]!=None):
            self.selectedIndices = sentinel2Info["selectedIndices"]
        else:
            print ("WARNING: No indices selected for interpretation. ",
                   "Use printAvailableIndicesS2() to print available indices")
        
        # Import and filter S2 SR.
        s2_sr_col = (ee.ImageCollection(sentinel2Info["collection"])
            .filterBounds(geometry)
            .filterDate(self.startDate, self.endDate)
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', self.CLOUD_FILTER)))

        # Import and filter s2cloudless.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(geometry)
            .filterDate(self.startDate, self.endDate))

        # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
        s2_sr_cld_col_eval = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
            'primary': s2_sr_col,
            'secondary': s2_cloudless_col,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index'
            })
        }))


        self.s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(self.add_cld_shdw_mask)

        self.s2_sr_cld_col_eval_disp = self.s2_sr_cld_col_eval_disp.map(algorithm = self.applyCloudMask)
        
        self.selectedIndicesRefined = []
        for index in self.selectedIndices:
            try: 
                self.s2_sr_cld_col_eval_disp = self.s2_sr_cld_col_eval_disp.map(algorithm = getattr(indices, f"add_{index}"))
                self.selectedIndicesRefined += [index]
            except:
                print(f"Failed to add index {index}. Invalid index name")
        
        
        self.selectedBandsRefined = [] 
        for band in self.selectedBands:
            if band in self.allbands :
                self.selectedBandsRefined = self.selectedBandsRefined + [band]
                        
        selectedBandsNIndices = []
        if(self.selectedBandsRefined==[]):
            selectedBandsNIndices = self.selectedIndicesRefined 
        elif (self.selectedIndicesRefined == []):
            selectedBandsNIndices = self.selectedBandsRefined
        else : 
            selectedBandsNIndices = self.selectedBands + self.selectedIndicesRefined 
            
        print("selectedBandsNIndices:", selectedBandsNIndices)
        self.s2_sr_cld_col_eval_disp = self.s2_sr_cld_col_eval_disp.select(selectedBandsNIndices)

        
        masksHandler = Masks(geometry,masks)
        masksHandler.calculateCombinedMask()
        self.s2_sr_cld_col_eval_disp = self.s2_sr_cld_col_eval_disp.map(algorithm = masksHandler.updateCombinedMask)
    
    
    def getFirst(self):
        return self.s2_sr_cld_col_eval_disp.first() 
  
    def getMedian(self):
        return self.s2_sr_cld_col_eval_disp.median() 

       
    def byMonth(self,i_year):
        self.s2_sr_cld_col_eval_disp = byMonth(i_year,self.s2_sr_cld_col_eval_disp) 

    def getCollection(self):
        return self.s2_sr_cld_col_eval_disp

    def getCollectionToBands(self):
        return self.s2_sr_cld_col_eval_disp.toBands()
    
    def getCollectionToListFlatten(self):
        return self.s2_sr_cld_col_eval_disp.toList().flatten()

    

In [6]:
print("Sentinel2b class imported")



Sentinel2b class imported
